# Procesado de resultados (+ logs PCSMOTE)
Notebook parametrizable para analizar múltiples **modelos** y **datasets**.

## Instrucciones
1. Editá la celda de **Configuración** con tus rutas y listas de modelos/datasets.
2. Ejecutá las celdas en orden.
3. Si activás `EXPORTAR_EXCEL`, se generará un .xlsx con hojas por (modelo,dataset) y logs encontrados.

**Convenciones de archivos**

- Resultados: `codigo/resultados/resultados_[Modelo].csv`  
- Logs PCSMOTE: `codigo/datasets/datasets_aumentados/pcsmote/logs/log_pcsmote_[dataset]_D[d]_R[r]_P[p].csv`

In [1]:
# === Configuración ===
from pathlib import Path

from pathlib import Path

# Directorio actual del notebook
BASE = Path.cwd()

# Subir un nivel (de notebooks → codigo)
ROOT = BASE.parent

PATH_RESULTADOS = ROOT / "resultados"
PATH_LOGS = ROOT / "datasets" / "datasets_aumentados" / "pcsmote" / "logs"

# Lista de modelos a estudiar (usa exactamente estos nombres para mapear a archivos):
# 'LogisticRegression' -> resultados_LogisticRegression.csv
# 'RandomForest'       -> resultados_RandomForest.csv
# 'SVM'                -> resultados_SVM.csv
MODELOS = ['LogisticRegression', 'RandomForest', 'SVM']  # editá lo que necesites

# Lista de datasets a estudiar
DATASETS = ['glass','ecoli','wdbc','heart','statlog+shuttle']  # agregá más: ['glass','ecoli','wdbc','heart','statlog+shuttle', ... ]

# Cantidad de filas top por (modelo,dataset)
TOP_N = 2

# Exportación a Excel (opcional)
EXPORTAR_EXCEL = True
RUTA_XLSX_SALIDA = ROOT / "resultados" / "estudio_de_resultados.xlsx"


In [2]:
# === Funciones utilitarias ===
import os,json, re
import pandas as pd
from collections import defaultdict
from IPython.display import display

PREFERENCIAS_METRICAS = [
    'gmean_macro', 'f1_macro', 'roc_auc_macro_ovr',
    'balanced_accuracy', 'kappa', 'mcc', 'accuracy', 'score'
]

# Acumuladores
bloques_por_modelo: dict[str, list[tuple[str, pd.DataFrame]]] = defaultdict(list)
logs_encontrados: list[tuple[str, pd.DataFrame]] = []

# --- Función para ajustar nombres de hoja ---
nombres_usados = set()

def alias_modelo(nombre: str) -> str:
    s = normalizar_str(nombre)
    if s == "randomforest":
        return "RF"
    if s == "logisticregression":
        return "LR"
    return nombre  # SVM u otros

def ajustar_nombre(nombre: str) -> str:
    nombre = nombre[:31]  # Excel no admite > 31 caracteres
    if nombre in nombres_usados:
        base = nombre[:28]  # dejar espacio para sufijo
        i = 1
        nuevo = f"{base}{i}"
        while nuevo in nombres_usados:
            i += 1
            nuevo = f"{base}{i}"
        nombre = nuevo
    nombres_usados.add(nombre)
    return nombre

def normalizar_str(x: str) -> str:
    return str(x).strip().lower()

def cargar_resultados(path_resultados: Path, modelos: list[str]) -> pd.DataFrame:
    dfs = []
    for m in modelos:
        ruta = path_resultados / f"resultados_{m}.csv"
        if not ruta.exists():
            print(f"[!] No encontré {ruta}, continúo…")
            continue
        try:
            df = pd.read_csv(ruta, encoding="utf-8")
        except UnicodeDecodeError:
            df = pd.read_csv(ruta, encoding="latin1")
        df.columns = [str(c).strip().replace(" ","_").replace("-","_").replace("/","_").lower() for c in df.columns]
        if "modelo" not in df.columns:
            df["modelo"] = m
        dfs.append(df)
    if not dfs:
        raise FileNotFoundError("No se cargó ningún CSV.")
    return pd.concat(dfs, ignore_index=True)

def elegir_metrica(resultados):
    dataset = resultados['dataset']
    densidad = resultados['densidad']
    riesgo = resultados['riesgo']
    pureza = resultados['pureza']
    
    for i in range(len(dataset)):
        archivo_log = f"log_pcsmote_{dataset.iloc[i]}_D{densidad.iloc[i]}_R{riesgo.iloc[i]}_P{pureza.iloc[i]}.csv"
        archivo_log_path = os.path.join(PATH_LOGS, archivo_log)
        
        # Verificar si el archivo existe
        if os.path.exists(archivo_log_path):
            print(f"Encontrado archivo de log: {archivo_log_path}")
        else:
            print(f"No encontrado archivo de log: {archivo_log_path}")
    
    return 'cv_f1_macro'

def col_dataset_name(df: pd.DataFrame) -> str:
    for cand in ('dataset','nombre_dataset','dataset_name','ds','nombre'):
        if cand in df.columns:
            return cand
    raise ValueError("No encuentro columna de dataset ('dataset' o 'nombre_dataset').")

def top_n_por_modelo_dataset(df: pd.DataFrame, modelo: str, dataset: str, metrica: str | None, n: int=2) -> pd.DataFrame:
    sub = df[df['modelo'].map(normalizar_str) == normalizar_str(modelo)].copy()
    cd = col_dataset_name(df)
    sub = sub[sub[cd].map(normalizar_str) == normalizar_str(dataset)]
    if sub.empty:
        return sub
    if metrica is None or metrica not in sub.columns:
        return sub.head(n)
    return sub.sort_values(metrica, ascending=False).head(n)

def detectar_pcsmote(row: pd.Series) -> bool:
    for c in ['tecnica','sobremuestreo','oversampler','sampler','tecnica_sobremuestreo']:
        if c in row.index and isinstance(row[c], str) and row[c] and row[c].strip().lower() == 'pcsmote':
            return True
    for c in ['dataset_file','mejor_configuracion','config']:
        if c in row.index and isinstance(row[c], str) and 'pcsmote' in row[c].lower():
            return True
    return False

def limpiar_num(x: str) -> str:
    """
    Convierte strings tipo '25.0' en '25', deja otros valores igual.
    """
    try:
        f = float(x)
        if f.is_integer():
            return str(int(f))
        return str(f)
    except:
        return str(x).strip()

def extraer_drps_desde_fila(row: pd.Series):
    # 1) columnas explícitas
    d = str(row.get('densidad','')).strip() or None
    r = str(row.get('riesgo','')).strip() or None
    p = str(row.get('pureza','')).strip() or None
    if d and r and p:
        return limpiar_num(d), limpiar_num(r), limpiar_num(p)

    # 2) JSON o texto en mejor_configuracion
    txt = str(row.get('mejor_configuracion','') or '')
    if txt:
        try:
            j = json.loads(txt)
            if isinstance(j, dict) and 'raw' in j and isinstance(j['raw'], str):
                try:
                    j2 = json.loads(j['raw'].replace("'", '"'))
                    j = j2
                except Exception:
                    pass
            d = j.get('densidad') or j.get('D') or j.get('densidad_pct')
            r = j.get('riesgo')   or j.get('R')
            p = j.get('pureza')   or j.get('P')
            if d and r and p:
                return str(d), str(r), str(p)
        except Exception:
            pass
        # Regex fallback
        mD = re.search(r"_D([^_]+)", txt)
        mR = re.search(r"_R([^_]+)", txt)
        mP = re.search(r"_P([^_./\\]+)", txt)
        if mD and mR and mP:
            return mD.group(1), mR.group(1), mP.group(1)

    # 3) dataset_file del estilo pcsmote_glass_D25_R25_Pproporcion_train.csv
    for c in ['dataset_file','train_file','test_file','archivo']:
        val = row.get(c)
        if isinstance(val, str) and val:
            mD = re.search(r"_D([^_]+)", val)
            mR = re.search(r"_R([^_]+)", val)
            mP = re.search(r"_P([^_./\\]+)", val)
            if mD and mR and mP:
                return mD.group(1), mR.group(1), mP.group(1)
    return None

def buscar_log_pcsmote(path_logs: Path, dataset: str, d: str, r: str, p: str):
    target = f"log_pcsmote_{dataset}_D{d}_R{r}_P{p}.csv".lower()
    if not path_logs.exists():
        return None
    for f in path_logs.rglob('*.csv'):
        if f.name.lower() == target:
            return f
    return None


In [3]:
# === Carga de resultados y selección de métrica ===
resultados = cargar_resultados(PATH_RESULTADOS, MODELOS)
METRICA = elegir_metrica(resultados)
print('Métrica elegida para ordenar (desc):', METRICA)

# Inferimos columna dataset una vez
COL_DATASET = None
for cand in ('dataset','nombre_dataset','dataset_name','ds','nombre'):
    if cand in resultados.columns:
        COL_DATASET = cand
        break
if COL_DATASET is None:
    raise ValueError("No encuentro columna de dataset ('dataset' o 'nombre_dataset').")

resultados.head(3)  # preview


No encontrado archivo de log: c:\Users\FamiliaNatelloMedina\Documents\UNLu\armado-tesina\codigo\datasets\datasets_aumentados\pcsmote\logs\log_pcsmote_glass_D25.0_R25.0_Pentropia.csv
No encontrado archivo de log: c:\Users\FamiliaNatelloMedina\Documents\UNLu\armado-tesina\codigo\datasets\datasets_aumentados\pcsmote\logs\log_pcsmote_glass_D25.0_R25.0_Pproporcion.csv
No encontrado archivo de log: c:\Users\FamiliaNatelloMedina\Documents\UNLu\armado-tesina\codigo\datasets\datasets_aumentados\pcsmote\logs\log_pcsmote_glass_D25.0_R50.0_Pentropia.csv
No encontrado archivo de log: c:\Users\FamiliaNatelloMedina\Documents\UNLu\armado-tesina\codigo\datasets\datasets_aumentados\pcsmote\logs\log_pcsmote_glass_D25.0_R50.0_Pproporcion.csv
No encontrado archivo de log: c:\Users\FamiliaNatelloMedina\Documents\UNLu\armado-tesina\codigo\datasets\datasets_aumentados\pcsmote\logs\log_pcsmote_glass_D25.0_R75.0_Pentropia.csv
No encontrado archivo de log: c:\Users\FamiliaNatelloMedina\Documents\UNLu\armado-tesi

,dataset,tipo,tecnica,densidad,riesgo,pureza,n_train,n_test,n_features,es_grande,...,cv_f1_macro,cv_balanced_accuracy,cv_mcc,cv_cohen_kappa,test_f1_macro,test_balanced_accuracy,test_mcc,test_cohen_kappa,search_time_sec,n_jobs_search
0,glass,aumentado,pcsmote,25.0,25.0,entropia,268,43,9,False,...,0.662340,0.723556,0.587720,0.579117,0.483100,0.646032,0.415484,0.382507,2.864,4
1,glass,aumentado,pcsmote,25.0,25.0,proporcion,171,43,9,False,...,0.602214,0.638124,0.468332,0.458554,0.603416,0.686508,0.531865,0.520223,0.721,4
2,glass,aumentado,pcsmote,25.0,50.0,entropia,268,43,9,False,...,0.674365,0.728556,0.596165,0.587568,0.454874,0.618254,0.380637,0.351050,0.484,4


In [4]:
# === Estudio por lotes (MODELOS x DATASETS) ===
from IPython.display import display
import pandas as pd

# Acumuladores para exportar
hojas_excel: dict[str, pd.DataFrame] = {}
logs_encontrados: list[tuple[str, pd.DataFrame]] = []

for modelo in MODELOS:
    for ds in DATASETS:
        print('='*90)
        print(f'Modelo: {modelo} | Dataset: {ds}')

        topN = top_n_por_modelo_dataset(resultados, modelo, ds, METRICA, n=TOP_N)

        # Fila base para este modelo/dataset
        base_row = resultados[
            (resultados['modelo'].map(normalizar_str) == normalizar_str(modelo)) &
            (resultados[COL_DATASET].map(normalizar_str) == normalizar_str(ds)) &
            (resultados['tipo'].map(normalizar_str) == 'base')
        ]

        # Concatenar topN + base (si existe)
        combinado = pd.concat([topN, base_row], ignore_index=True) if not base_row.empty else topN

        if combinado.empty:
            print(' -> Sin filas para esta combinación.')
            continue

        display(combinado)

        # Guardar bloque en la hoja del MODELO (una hoja por modelo; múltiples datasets)
        bloques_por_modelo[modelo].append((ds, combinado))

        # ---- Logs PCSMOTE (igual que antes) ----
        for idx, row in topN.iterrows():
            if detectar_pcsmote(row):
                drp = extraer_drps_desde_fila(row)
                if not drp:
                    print(f'   [Fila {idx}] PCSMOTE detectado pero no pude extraer (D,R,P).')
                    continue
                d, r, p = drp
                ds_norm = str(row[COL_DATASET]).strip().lower()
                log_path = buscar_log_pcsmote(PATH_LOGS, ds_norm, str(d), str(r), str(p))
                if log_path is None or not log_path.exists():
                    print(f'   [!] No se encontró log: log_pcsmote_{ds_norm}_D{d}_R{r}_P{p}.csv en {PATH_LOGS}')
                    continue
                try:
                    log_df = pd.read_csv(log_path)
                    print(f'   [+] Log leído: {log_path.name}  (filas={len(log_df)})')
                    display(log_df.head(10))

                    mstr = normalizar_str(modelo)
                    modelo_norm = "RF" if mstr == "randomforest" else ("LR" if mstr == "logisticregression" else modelo)
                    hoja_nombre = f'log_{modelo_norm}_{ds_norm}_{d}_{r}_{p}'

                    logs_encontrados.append((hoja_nombre, log_df))
                except Exception as e:
                    print(f'   [x] Error leyendo {log_path.name}: {e}')

Modelo: LogisticRegression | Dataset: glass


,dataset,tipo,tecnica,densidad,riesgo,pureza,n_train,n_test,n_features,es_grande,...,cv_f1_macro,cv_balanced_accuracy,cv_mcc,cv_cohen_kappa,test_f1_macro,test_balanced_accuracy,test_mcc,test_cohen_kappa,search_time_sec,n_jobs_search
0,glass,aumentado,pcsmote,50.0,25.0,entropia,268,43,9,False,...,0.730344,0.770194,0.674097,0.664657,0.596265,0.746032,0.541635,0.515573,0.493,4
1,glass,aumentado,pcsmote,50.0,75.0,entropia,268,43,9,False,...,0.703746,0.741737,0.634411,0.627837,0.596265,0.746032,0.541635,0.515573,0.491,4
2,glass,base,base,NaN,NaN,NaN,149,65,9,False,...,0.549947,0.603114,0.468413,0.445878,0.603764,0.694272,0.546051,0.512957,0.641,4


   [+] Log leído: log_pcsmote_glass_D50_R25_Pentropia.csv  (filas=6)


,dataset,clase,train_original,objetivo_balance,estado,motivo_sin_sinteticas,muestras_sinteticas_generadas,total_original,total_resampled,ratio_original,...,umbral_densidad,umbral_entropia,percentil_densidad,percentil_riesgo,criterio_pureza,tecnica_sobremuestreo,factor_equilibrio,random_state,modo_espacial,timestamp
0,glass,1,56,48,no se sobremuestrea,sin_faltante(actual>=objetivo),0,171,171,0.327485,...,NaN,NaN,50.0,25.0,entropia,PCSMOTE,0.8,42,2d,2025-08-20T00:20:20.002049
1,glass,2,61,48,no se sobremuestrea,sin_faltante(actual>=objetivo),0,171,171,0.356725,...,NaN,NaN,50.0,25.0,entropia,PCSMOTE,0.8,42,2d,2025-08-20T00:20:20.002049
2,glass,3,14,48,sobremuestreada,ok,34,171,205,0.081871,...,1.0,0.721928,50.0,25.0,entropia,PCSMOTE,0.8,42,2d,2025-08-20T00:20:20.046713
3,glass,5,10,48,sobremuestreada,ok,38,171,243,0.058480,...,0.0,0.908695,50.0,25.0,entropia,PCSMOTE,0.8,42,2d,2025-08-20T00:20:20.072602
4,glass,6,7,48,sobremuestreada,insuficientes_filtradas,0,171,243,0.040936,...,0.4,0.970951,50.0,25.0,entropia,PCSMOTE,0.8,42,2d,2025-08-20T00:20:20.084190
5,glass,7,23,48,sobremuestreada,ok,25,171,268,0.134503,...,1.0,0.000000,50.0,25.0,entropia,PCSMOTE,0.8,42,2d,2025-08-20T00:20:20.125995


   [+] Log leído: log_pcsmote_glass_D50_R75_Pentropia.csv  (filas=6)


...

   [+] Log leído: log_pcsmote_heart_D75_R25_Pentropia.csv  (filas=5)


,dataset,clase,train_original,objetivo_balance,estado,motivo_sin_sinteticas,muestras_sinteticas_generadas,total_original,total_resampled,ratio_original,...,umbral_densidad,umbral_entropia,percentil_densidad,percentil_riesgo,criterio_pureza,tecnica_sobremuestreo,factor_equilibrio,random_state,modo_espacial,timestamp
0,heart,0,128,102,no se sobremuestrea,sin_faltante(actual>=objetivo),0,237,237,0.540084,...,NaN,NaN,75.0,25.0,entropia,PCSMOTE,0.8,42,2d,2025-08-20T00:20:24.174921
1,heart,1,43,102,sobremuestreada,ok,59,237,296,0.181435,...,0.1,0.970951,75.0,25.0,entropia,PCSMOTE,0.8,42,2d,2025-08-20T00:20:24.259831
2,heart,2,28,102,sobremuestreada,ok,74,237,370,0.118143,...,0.0,0.970951,75.0,25.0,entropia,PCSMOTE,0.8,42,2d,2025-08-20T00:20:24.377767
3,heart,3,28,102,sobremuestreada,ok,74,237,444,0.118143,...,0.0,0.970951,75.0,25.0,entropia,PCSMOTE,0.8,42,2d,2025-08-20T00:20:24.433553
4,heart,4,10,102,sobremuestreada,ok,92,237,536,0.042194,...,0.0,0.541446,75.0,25.0,entropia,PCSMOTE,0.8,42,2d,2025-08-20T00:20:24.468936


   [+] Log leído: log_pcsmote_heart_D75_R50_Pproporcion.csv  (filas=5)


,dataset,clase,train_original,objetivo_balance,estado,motivo_sin_sinteticas,muestras_sinteticas_generadas,total_original,total_resampled,ratio_original,...,umbral_densidad,umbral_entropia,percentil_densidad,percentil_riesgo,criterio_pureza,tecnica_sobremuestreo,factor_equilibrio,random_state,modo_espacial,timestamp
0,heart,0,128,102,no se sobremuestrea,sin_faltante(actual>=objetivo),0,237,237,0.540084,...,NaN,NaN,75.0,50.0,proporcion,PCSMOTE,0.8,42,2d,2025-08-20T00:20:25.103324
1,heart,1,43,102,sobremuestreada,ok,59,237,296,0.181435,...,0.1,NaN,75.0,50.0,proporcion,PCSMOTE,0.8,42,2d,2025-08-20T00:20:25.134566
2,heart,2,28,102,sobremuestreada,ok,74,237,370,0.118143,...,0.0,NaN,75.0,50.0,proporcion,PCSMOTE,0.8,42,2d,2025-08-20T00:20:25.181549
3,heart,3,28,102,sobremuestreada,ok,74,237,444,0.118143,...,0.0,NaN,75.0,50.0,proporcion,PCSMOTE,0.8,42,2d,2025-08-20T00:20:25.204795
4,heart,4,10,102,sobremuestreada,insuficientes_filtradas,0,237,444,0.042194,...,0.0,NaN,75.0,50.0,proporcion,PCSMOTE,0.8,42,2d,2025-08-20T00:20:25.204795


Modelo: SVM | Dataset: statlog+shuttle
 -> Sin filas para esta combinación.


In [5]:
import pandas as pd

EXPORTAR_EXCEL = True

if EXPORTAR_EXCEL:
    RUTA_XLSX_SALIDA.parent.mkdir(parents=True, exist_ok=True)
    with pd.ExcelWriter(RUTA_XLSX_SALIDA, engine="xlsxwriter") as xw:
        # Formatos
        fmt_border = xw.book.add_format({'border': 1})
        fmt_wrap   = xw.book.add_format({'text_wrap': True, 'valign': 'top', 'border': 1})
        fmt_title  = xw.book.add_format({'bold': True})

        # ===== Hojas TOP por MODELO (una hoja por modelo; múltiples datasets) =====
        for modelo, bloques in bloques_por_modelo.items():
            sheet_name = ajustar_nombre(f"top+{alias_modelo(modelo)}")
            # Worksheet se crea al primer to_excel
            # Para calcular anchos: concateno todos los bloques (solo para sizing)
            df_all = pd.concat([df for _, df in bloques], ignore_index=True) if bloques else pd.DataFrame()

            start_row = 0
            for ds, df in bloques:
                # Rótulo de dataset
                ws = xw.book.add_worksheet(sheet_name) if sheet_name not in xw.sheets else xw.sheets[sheet_name]
                if start_row == 0 and sheet_name not in xw.sheets:
                    # si recién se creó, registrar
                    xw.sheets[sheet_name] = ws
                else:
                    ws = xw.sheets[sheet_name]

                ws.write(start_row, 0, f"Dataset: {ds}", fmt_title)
                start_row += 1

                # Escribir bloque con cabecera
                df.to_excel(xw, sheet_name=sheet_name, startrow=start_row, index=False, header=True)
                start_row += len(df) + 2  # una fila en blanco entre bloques

            # Ajustes de columnas (sobre df_all)
            if not df_all.empty:
                ws = xw.sheets[sheet_name]
                # Columna N (índice 13)
                ws.set_column(13, 13, 38, fmt_wrap)
                # Resto auto-ajuste + borde
                for idx, col in enumerate(df_all.columns):
                    if idx == 13:
                        continue
                    max_len = max([len(str(col))] + [len(str(v)) for v in df_all[col].astype(str)])
                    ancho = min(max_len + 2, 50)
                    ws.set_column(idx, idx, ancho, fmt_border)

        # ===== Hojas de LOGS (misma lógica que ya usabas) =====
        for name, logdf in logs_encontrados:
            sheet = ajustar_nombre(name)
            logdf.to_excel(xw, index=False, sheet_name=sheet)
            ws = xw.sheets[sheet]
            for idx, col in enumerate(logdf.columns):
                max_len = max([len(str(col))] + [len(str(v)) for v in logdf[col].astype(str)])
                ancho = min(max_len + 2, 50)
                ws.set_column(idx, idx, ancho, fmt_border)

    print(f"[i] Exportado a: {RUTA_XLSX_SALIDA.resolve()}")
else:
    print("Exportación a Excel desactivada (EXPORTAR_EXCEL=False).")

[i] Exportado a: C:\Users\FamiliaNatelloMedina\Documents\UNLu\armado-tesina\codigo\resultados\estudio_de_resultados.xlsx
